In [22]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta, date
import concurrent.futures

In [21]:
DOMAIN = "http://news.ltn.com.tw"

dates = []
d_format = "%Y%m%d"
date = datetime.strptime("20140101", d_format)
delta = timedelta(days=1)
today = datetime.strptime(date.today().strftime(d_format), d_format)
while date <= today:
    dates.append(date.strftime("%Y%m%d"))
    date += delta

In [ ]:
def worker(date):
    resp = requests.get("http://news.ltn.com.tw/list/newspaper/politics/{}".format(date))
    soup = BeautifulSoup(resp.text, "html5lib")
    urls = [a.get("href") for a in soup.select("ul.list > li > a.tit")]
    
    for path in paths:
        url = DOMAIN + path
        c_resp = requests.get(url)
        soup = BeautifulSoup(c_resp.text, "html5lib")
        
        news = {}
        news["id"]       = url.split("/")[-1]
        news["title"]    = soup.select("div.articlebody > h1")[0].text.replace("\n", "").replace("\t", "")
        news["datetime"] = soup.select("div.text > span")[0].text
        content = "".join([p.text.strip() for p in soup.select("div.text > p")])
        news["content"]  = re.findall("〔.+〕(.+)", content)[0]
        news["hash"]     = hash(c_resp.text)
        news["media"]    = "自由時報"
        news["category"] = "politics"
        news["url"] = url
        
        # store to mongoDB

In [ ]:
def main():
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        executor.map(worker, dates)

In [ ]:
if __name__ == "__main__":
    main()